<a href="https://colab.research.google.com/github/tanmalaker/GEE/blob/main/NAIP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing Libraries
!pip install geemap
import ee
import geemap

# Authenticate and initialize............
ee.Authenticate()
ee.Initialize(project='ee-malakerku')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 14.3 MB/s eta 0:00:00


In [2]:
# Load and mosaic NAIP images (latest 2022–2024)
# The area is in Brazoria Texas
roi = ee.Geometry.Polygon([[
  [-95.63, 29.15],
  [-95.67, 29.15],
  [-95.67, 29.11],
  [-95.60, 29.11]
]])

naip = ee.ImageCollection('USDA/NAIP/DOQQ') \
          .filterBounds(roi) \
          .filterDate('2020-01-01', '2025-12-31') \
          .sort('system:time_start', False) \
          .mosaic() \
          .clip(roi)


# Create map and center it on ROI
Map = geemap.Map()
Map.centerObject(roi, 14)

# Add layers
Map.addLayer(naip, {'bands': ['R', 'G', 'B'], 'min': 0, 'max': 255}, 'NAIP 2020–2025')
Map.addLayer(roi, {'color': 'red'}, 'ROI')
Map

Map(center=[29.128183849814178, -95.64181688585069], controls=(WidgetControl(options=['position', 'transparent…

In [ ]:
# Set export task includes NIR= 4bands, for the building footprint extraction we need RGB not 4 bands togather
task = ee.batch.Export.image.toDrive(
    image=naip,
    description='NAIP_Mosaic_Export',
    folder='GEE_Exports',
    fileNamePrefix='NAIP_Brazoria',
    region=roi,
    scale=0.6,  # native resolution
    maxPixels=1e13
)

# Start the export task
task.start()

# Check status (optional)
import time
print("Export started... Checking status every 30 seconds.")
while task.active():
    print(task.status())
    time.sleep(30)

print("Done.")
##Result will be a 4 band image, but show a greyscale in bareland and higher refelectance values. Solution is download RGB excluding NIR (3 band image)

Export started... Checking status every 30 seconds.
{'state': 'READY', 'description': 'NAIP_Mosaic_Export', 'priority': 100, 'creation_timestamp_ms': 1751488690034, 'update_timestamp_ms': 1751488690034, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': 'VAV7WG6YEXJMA7SVQKC5HU63', 'name': 'projects/ee-malakerku/operations/VAV7WG6YEXJMA7SVQKC5HU63'}
{'state': 'RUNNING', 'description': 'NAIP_Mosaic_Export', 'priority': 100, 'creation_timestamp_ms': 1751488690034, 'update_timestamp_ms': 1751488719700, 'start_timestamp_ms': 1751488697723, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'VAV7WG6YEXJMA7SVQKC5HU63', 'name': 'projects/ee-malakerku/operations/VAV7WG6YEXJMA7SVQKC5HU63'}
{'state': 'RUNNING', 'description': 'NAIP_Mosaic_Export', 'priority': 100, 'creation_timestamp_ms': 1751488690034, 'update_timestamp_ms': 1751488739694, 'start_timestamp_ms': 1751488697723, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': 'VAV7WG6YEXJMA7SVQKC5HU63', 'name': 'projects/ee-malakerku/operat

In [ ]:
# Select RGB only
naip_rgb = naip.select(['R', 'G', 'B'])

# Optional: Visualize for better-looking output
vis = naip_rgb.visualize(min=0, max=255)

# Export visualized image to Drive
task = ee.batch.Export.image.toDrive(
    image=vis,
    description='NAIP_Export_RGB',
    folder='LSBP',
    fileNamePrefix='BrazoriaBld_rgb',
    region=roi.bounds(),
    scale=0.6,# native resolution
    maxPixels=1e13
)
task.start()
# Check status (optional)
import time
print("Export started... Checking status every 30 seconds.")
while task.active():
    print(task.status())
    time.sleep(30)

print("Done.")

Export started... Checking status every 30 seconds.
{'state': 'READY', 'description': 'NAIP_Export_RGB', 'priority': 100, 'creation_timestamp_ms': 1751489111267, 'update_timestamp_ms': 1751489111267, 'start_timestamp_ms': 0, 'task_type': 'EXPORT_IMAGE', 'id': '5ZCTNAKGAZACRNLFE7YAHDRG', 'name': 'projects/ee-malakerku/operations/5ZCTNAKGAZACRNLFE7YAHDRG'}
{'state': 'RUNNING', 'description': 'NAIP_Export_RGB', 'priority': 100, 'creation_timestamp_ms': 1751489111267, 'update_timestamp_ms': 1751489138670, 'start_timestamp_ms': 1751489118025, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': '5ZCTNAKGAZACRNLFE7YAHDRG', 'name': 'projects/ee-malakerku/operations/5ZCTNAKGAZACRNLFE7YAHDRG'}
{'state': 'RUNNING', 'description': 'NAIP_Export_RGB', 'priority': 100, 'creation_timestamp_ms': 1751489111267, 'update_timestamp_ms': 1751489158666, 'start_timestamp_ms': 1751489118025, 'task_type': 'EXPORT_IMAGE', 'attempt': 1, 'id': '5ZCTNAKGAZACRNLFE7YAHDRG', 'name': 'projects/ee-malakerku/operations/5ZCT